In [1]:
from datetime import datetime, timedelta, date
from calendar import monthrange
# from pytz import timezone
import pandas as pd
import numpy as np
#import feather
import datetime 
import pytz
import csv
import os
import seaborn as sns
import matplotlib.pyplot as plt
import logging

import xgboost as xgb
import time
from sklearn.pipeline import *

from math import sqrt
from numpy import loadtxt
from itertools import product
from tqdm import tqdm
from sklearn import preprocessing
from xgboost import plot_tree

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer

kernel_with_output = False


from utils import *

%matplotlib inline

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [2]:
# https://www.kaggle.com/szhou42/predict-future-sales-top-11-solution

# scp jini:/home/ubuntu/future-store-sales/submissions/xgb_base_submission.csv ~/Documents/reading_training/future-store-sales/submissions/.

In [2]:
DATA_FOLDER = './data/'
ADD_RESOURCES = './res/'

sales_train    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
test = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv.gz')) 
sample_submission = pd.read_csv(os.path.join(DATA_FOLDER, 'sample_submission.csv.gz'))


date_df = pd.read_pickle(os.path.join(ADD_RESOURCES, "russian_holidays.pkl"))
geography = pd.read_csv(os.path.join(ADD_RESOURCES, 'geography.csv'))
#holidays = feather.read_dataframe(os.path.join(ADD_RESOURCES, 'hol.feather'))

In [10]:
# For every month we create a grid from all shops/items combinations from that month

grid = []
for block_num in sales_train.date_block_num.unique():
    cur_shops = sales_train[sales_train.date_block_num == block_num]['shop_id'].unique()
    cur_items = sales_train[sales_train.date_block_num == block_num]['item_id'].unique()
    
    # cartesian product of shops, items with the block number 
    l = list( product(*[cur_shops, cur_items, [block_num]]) ) # sample l[0:2] -> [(45, 13315, 33), (45, 13881, 33)]
    
    grid.append(np.array(l , dtype='int32'))
    # np-array looks like this:
        #     array([[   45, 13315,    33],
        #        [   45, 13880,    33],
        #        [   45, 13881,    33],
        #        ...,
        #        [   21,  7640,    33],
        #        [   21,  7632,    33],
        #        [   21,  7440,    33]], dtype=int32)
    # right now, 'grid' -> is a list of np-arrays

index_cols = ['shop_id', 'item_id', 'date_block_num']
# np.vstack is used to unpack the rows from all np.arrays and put into a single array
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32) 

In [11]:
# Aggregations
sales_train['item_cnt_day'] = sales_train['item_cnt_day'].clip(0,20) # we say no-item sells more than 20 everyday
groups = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'])
trainset = groups.agg({'item_cnt_day':'sum', 'item_price':'mean'}).reset_index()
trainset = trainset.rename(columns = {'item_cnt_day' : 'item_cnt_month'})
trainset = trainset.rename(columns = {'item_price' : 'avg_item_price'})

trainset['item_cnt_month'] = trainset['item_cnt_month'].clip(0,20)

trainset = pd.merge(grid,trainset,how='left',on=index_cols)
trainset.item_cnt_month = trainset.item_cnt_month.fillna(0)

# Get category id
trainset = pd.merge(trainset, items[['item_id', 'item_category_id']], on = 'item_id')
trainset.to_csv('trainset_with_grid.csv', index_label=False)

trainset.head()

,shop_id,item_id,date_block_num,item_cnt_month,avg_item_price,item_category_id
0,59,22154,0,1.0,999.0,37
1,25,22154,0,5.0,999.0,37
2,24,22154,0,1.0,999.0,37
3,23,22154,0,0.0,NaN,37
4,19,22154,0,0.0,NaN,37


In [3]:
trainset = pd.read_csv("./trainset_with_grid.csv")
trainset.head()

,Unnamed: 0,shop_id,item_id,date_block_num,item_cnt_month,item_price,item_category_id
0,0,59,22154,0,1.0,999.0,37
1,1,25,22154,0,5.0,999.0,37
2,2,24,22154,0,1.0,999.0,37
3,3,23,22154,0,0.0,NaN,37
4,4,19,22154,0,0.0,NaN,37


In [4]:
np.sort(trainset.date_block_num.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33],
      dtype=int64)

In [5]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


#### Benchmark submission

In [9]:
last_month = trainset[trainset.date_block_num==33]
last_month = last_month.groupby(['shop_id', 'item_id']).agg({'item_cnt_month':'mean'}).reset_index()

submission = test.merge(last_month,on=['shop_id', 'item_id'] , how='left', indicator=True)
submission['item_cnt_month'] = submission.item_cnt_month.fillna(0)
print(submission.shape)

submission.drop(columns=['shop_id', 'item_id', '_merge'], inplace=True)
submission = submission.set_index('ID')

print(submission.head())
submission.to_csv("./submissions/benchmark_submission.csv")

(214200, 5)


#### Using baseline features with xgboost

In [11]:
baseline_features = ['shop_id', 'item_id', 'item_category_id', 'date_block_num']
target = 'item_cnt_month'

In [12]:
# prepare test
merged_test = test.merge(items[['item_id' , 'item_category_id']], on='item_id', how='left')
merged_test['date_block_num'] = 34

merged_test.head()

,ID,shop_id,item_id,item_category_id,date_block_num
0,0,5,5037,19,34
1,1,5,5320,55,34
2,2,5,5233,19,34
3,3,5,5232,23,34
4,4,5,5268,20,34


In [13]:
base_pipeline = make_pipeline(
    ColumnSelector(baseline_features),
    xgb.XGBRegressor(max_depth = 10, min_child_weight=0.5, subsample = 1, eta = 0.3, num_round = 50, seed = 1, eval_metric='rmse')
)

In [11]:
base_pipeline.fit(trainset, trainset.item_cnt_month)

/home/ubuntu/miniconda3/envs/py36/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/ubuntu/miniconda3/envs/py36/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Pipeline(memory=None,
     steps=[('columnselector', ColumnSelector(columns=['shop_id', 'item_id', 'item_category_id', 'date_block_num'])), ('xgbregressor', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eta=0.3, eval_metric='rmse', gamma=0,
       importance_type='gain', learni...m_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=1, silent=True, subsample=1))])

In [20]:
submission = test
submission['item_cnt_month'] = base_pipeline.predict(merged_test)
submission = submission.set_index('ID')
submission.drop(columns=['item_id', 'shop_id'], inplace=True)
submission.to_csv("./submissions/xgb_base_submission.csv")

###### Build a train_test_set for combined feature engineering

In [14]:
train_set = trainset[baseline_features + [target]]

test_set = test.merge(items[['item_id' , 'item_category_id']], on='item_id', how='left')
test_set['date_block_num'] = 34
test_set ['item_cnt_month'] = -1
test_set.set_index('ID', inplace=True)

train_test_set = pd.concat([train_set, test_set], axis=0)

In [15]:
train_test_set.head()

,shop_id,item_id,item_category_id,date_block_num,item_cnt_month
0,59,22154,37,0,1.0
1,25,22154,37,0,5.0
2,24,22154,37,0,1.0
3,23,22154,37,0,0.0
4,19,22154,37,0,0.0


In [22]:
print(item_categories.shape)
item_categories.head()

(84, 2)


,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [35]:
from py_translator import Translator

l_cat = list(item_categories.item_category_name)
l_cat_en = []
l_cat_tr = Translator().translate(l_cat, dest='en')
for tr in l_cat_tr:
    l_cat_en.append(tr.text)
print(l_cat_en)

['PC - Headsets / Headphones', 'Accessories - PS2', 'Accessories - PS3', 'Accessories - PS4', 'Accessories - PSP', 'Accessories - PSVita', 'Accessories - XBOX 360', 'Accessories - XBOX ONE', 'Tickets (Digital)', 'Delivery of goods', 'Game Consoles - PS2', 'Game Consoles - PS3', 'Game Consoles - PS4', 'Gaming Consoles - PSP', 'Game Consoles - PSVita', 'Gaming Consoles - XBOX 360', 'Gaming Consoles - XBOX ONE', 'Game Consoles - Other', 'Games - PS2', 'Games - PS3', 'Games - PS4', 'Games - PSP', 'Games - PSVita', 'Games - XBOX 360', 'Games - XBOX ONE', 'Games - Accessories for games', 'Android Games - Digital', 'MAC Games - Digit', 'PC Games - Additional Editions', 'PC Games - Collectible Editions', 'PC Games - Standard Editions', 'PC Games - Digital', 'Payment cards (Cinema, Music, Games)', 'Payment Cards - Live!', 'Payment Cards - Live! (Numeral)', 'Payment Cards - PSN', 'Payment Cards - Windows (Digital)', 'Cinema - Blu-Ray', 'The Movie - Blu-Ray 3D', 'Cinema - Blu-Ray 4K', 'Cinema - D

In [39]:
for l,i in enumerate(l_cat_en):
    if l in replacements:
        l_cat_en[i] = replacements[l]

In [42]:
l_cat_en[0] = 'PC Headsets / Headphones'

In [51]:
catgs = pd.DataFrame({'catg_en':l_cat_en})

In [57]:
catgs['catg_en_b'] = catgs.catg_en.apply(lambda x: x.split(" -")[0])

In [66]:
catgs.catg_en_b[13:17] = 'Game Consoles'

In [78]:
list(catgs.catg_en_b)

['PC Headsets / Headphones',
 'Accessories',
 'Accessories',
 'Accessories',
 'Accessories',
 'Accessories',
 'Accessories',
 'Accessories',
 'Tickets (Digital)',
 'Delivery of goods',
 'Game Consoles',
 'Game Consoles',
 'Game Consoles',
 'Game Consoles',
 'Game Consoles',
 'Game Consoles',
 'Game Consoles',
 'Game Consoles',
 'Games',
 'Games',
 'Games',
 'Games',
 'Games',
 'Games',
 'Games',
 'Games',
 'Android Games',
 'MAC Games',
 'PC Games',
 'PC Games',
 'PC Games',
 'PC Games',
 'Payment cards (Cinema, Music, Games)',
 'Payment Cards',
 'Payment Cards',
 'Payment Cards',
 'Payment Cards',
 'Cinema',
 'The Movie',
 'Cinema',
 'Cinema',
 'Cinema',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Книги',
 'Books',
 'Books',
 'Music',
 'Music',
 'Music',
 'Music',
 'Music',
 'Music',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Programs',
 'Programs',


In [74]:
list(catgs.catg_en)

['PC Headsets / Headphones',
 'Accessories - PS2',
 'Accessories - PS3',
 'Accessories - PS4',
 'Accessories - PSP',
 'Accessories - PSVita',
 'Accessories - XBOX 360',
 'Accessories - XBOX ONE',
 'Tickets (Digital)',
 'Delivery of goods',
 'Game Consoles - PS2',
 'Game Consoles - PS3',
 'Game Consoles - PS4',
 'Gaming Consoles - PSP',
 'Game Consoles - PSVita',
 'Gaming Consoles - XBOX 360',
 'Gaming Consoles - XBOX ONE',
 'Game Consoles - Other',
 'Games - PS2',
 'Games - PS3',
 'Games - PS4',
 'Games - PSP',
 'Games - PSVita',
 'Games - XBOX 360',
 'Games - XBOX ONE',
 'Games - Accessories for games',
 'Android Games - Digital',
 'MAC Games - Digit',
 'PC Games - Additional Editions',
 'PC Games - Collectible Editions',
 'PC Games - Standard Editions',
 'PC Games - Digital',
 'Payment cards (Cinema, Music, Games)',
 'Payment Cards - Live!',
 'Payment Cards - Live! (Numeral)',
 'Payment Cards - PSN',
 'Payment Cards - Windows (Digital)',
 'Cinema - Blu-Ray',
 'The Movie - Blu-Ray 3D'

In [56]:
# Fix category
item_cat = pd.read_csv('data/item_categories.csv')


for ind in range(0,1):
    l_cat[ind] = 'PC Headsets / Headphones'
for ind in range(1,8):
    l_cat[ind] = 'Access'
l_cat[8] = 'Tickets (figure)'
l_cat[9] = 'Delivery of goods'
for ind in range(10,18):
    l_cat[ind] = 'Consoles'
for ind in range(18,25):
    l_cat[ind] = 'Consoles Games'
l_cat[25] = 'Accessories for games'
for ind in range(26,28):
    l_cat[ind] = 'phone games'
for ind in range(28,32):
    l_cat[ind] = 'CD games'
for ind in range(32,37):
    l_cat[ind] = 'Card'
for ind in range(37,43):
    l_cat[ind] = 'Movie'
for ind in range(43,55):
    l_cat[ind] = 'Books'
for ind in range(55,61):
    l_cat[ind] = 'Music'
for ind in range(61,73):
    l_cat[ind] = 'Gifts'
for ind in range(73,79):
    l_cat[ind] = 'Soft'
for ind in range(79,81):
    l_cat[ind] = 'Office'
for ind in range(81,83):
    l_cat[ind] = 'Clean'
l_cat[83] = 'Elements of a food'

In [34]:

lb = preprocessing.LabelEncoder()
item_cat['item_category_id_fix'] = lb.fit_transform(l_cat)
item_cat['item_category_name_fix'] = l_cat
train_test_set = train_test_set.merge(item_cat[['item_category_id', 'item_category_id_fix']], on = 'item_category_id', how = 'left')
_ = train_test_set.drop(['item_category_id'],axis=1, inplace=True)
train_test_set.rename(columns = {'item_category_id_fix':'item_category_id'}, inplace = True)

_ = item_cat.drop(['item_category_id'],axis=1, inplace=True)
_ = item_cat.drop(['item_category_name'],axis=1, inplace=True)

item_cat.rename(columns = {'item_category_id_fix':'item_category_id'}, inplace = True)
item_cat.rename(columns = {'item_category_name_fix':'item_category_name'}, inplace = True)
item_cat = item_cat.drop_duplicates()
item_cat.index = np.arange(0, len(item_cat))

NameError: name 'item_cat' is not defined

###### Idea 0: Add previous shop/item sales as feature (Lag feature)

###### Idea 1: Add previous item sales as feature (Lag feature)

###### Idea 2: Add previous shop/item price as feature (Lag feature)

###### Idea 3: Add previous item price as feature (Lag feature)

###### Idea 6: Number of month from last sale of shop/item (Use info from past)

###### Idea 7: Number of month from last sale of item(Use info from past)

###### Idea 8: Item name (Tfidf text feature)

###### Cross Validation (Hyperparameter tuning)